# Lab 3: Extending Logistic Regression

#### *Harrison Noble & Henry Lambson*

## 1. Preparation & Overview

### 1.1 Business Understanding

In this lab we will be using the same wine quality dataset [Ref 1] we used in Lab 1 [Ref 2]. To jog your memory, the wine quality dataset contains 1599 red wine samples and 4898 white wine samples, with 6497 samples in total. There are 12 unique features including the quality value. Each feature is aside from the quality is numerical, with the quality being categorical.

The task for this lab is to create custom logistic regression classifiers and optimization techniques for predicting the quality of both red and white wines based on their physical/chemical attributes. The use-case for a classifier of this nature would be to create an initial screening process for new wines to help wine tasters filter out high quality wines. This would allow wine testers to only test high quality wines that made it through our filter in order to save time and resources that would have been spent on lower quality wines not fit for market. __To create an effective wine quality classifier which could eventually be used as a filtering software, we would like to have a prediction accuracy of 90% or higher that minimizes false positives. Minimizing false positives is ideal for this task so that lower quality wines are more likely to get screened out and only high quality wines make it through. This would lead wine testers/tasters to have a higher level of trust in our algorithm.__ 

__The model created from this dataset would mostly used for offline analysis.__ Because wine testers/tasters do not necessarily need to know the predicted quality of the wine that is produced instantly, this model can be used for offline analysis. One potential application of the model would be doing a weekly analysis of all wines submitted during that week. In this case, wine testers/tasters would gain insights on the high quality wines to potentially test further. 

One change we are making for this lab is keeping the white and red wine datasets seperate. Based on our analysis from Lab 1, we found that some attribute values that make white wine good would make red wine worse and vice versa. Therefore, we will train and test our regression models on each dataset seperately to see if the model works better on one or the other (or both!)

### 1.2 Defining and Preparing Data

In [1]:
import pandas as pd
import numpy as np

#This code chunk was taken from our Lab 1 assignment and edited for this lab.
# load red and white wine datasets into pandas
df_red = pd.read_csv('./winequality-red.csv', sep=';')
df_white = pd.read_csv('./winequality-white.csv', sep=';')

#print number of rows in each df to confirm all data is loaded in
print('Length of red wine dataset:', df_red.shape[0])
print('Length of white wine dataset:', df_white.shape[0])

df_red.head()

Length of red wine dataset: 1599
Length of white wine dataset: 4898


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [2]:
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Here we are just double checking the data was loaded into the dataframes correctly. One thing we are doing different in this lab is keeping the white and red wines separate. This was stated in section 1.1 but we just wanted to clarify. 

In [3]:
from scipy import stats

to_use = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
          'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
          'pH', 'sulphates', 'alcohol']

#remove outliers with absolute value z-scores 3 or higher
df_red = df_red[(np.abs(stats.zscore(df_red[to_use])) < 3).all(axis=1)]
df_white = df_white[(np.abs(stats.zscore(df_white[to_use])) < 3).all(axis=1)]

print('Length of red wine dataset:', df_red.shape[0])
print('Length of white wine dataset:', df_white.shape[0])

df_red.describe()

Length of red wine dataset: 1458
Length of white wine dataset: 4502


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000
mean,8.312551,0.524050,0.265281,2.388717,0.081531,15.089849,43.660494,0.996718,3.316152,0.642414,10.417798,5.646776
std,1.647635,0.169451,0.191271,0.865307,0.021218,9.317669,29.414615,0.001718,0.141052,0.129753,1.021649,0.801119
min,5.000000,0.120000,0.000000,1.200000,0.038000,1.000000,6.000000,0.991500,2.880000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,21.000000,0.995600,3.220000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,36.000000,0.996700,3.315000,0.620000,10.200000,6.000000
75%,9.200000,0.635000,0.420000,2.600000,0.089000,21.000000,58.000000,0.997800,3.400000,0.720000,11.100000,6.000000
max,13.500000,1.040000,0.790000,6.700000,0.226000,47.000000,145.000000,1.002200,3.750000,1.160000,13.600000,8.000000


In [4]:
df_white.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000,4502.000000
mean,6.840749,0.271465,0.326513,6.414705,0.043153,34.821302,137.533319,0.993964,3.188256,0.485426,10.540115,5.912261
std,0.786885,0.086082,0.101038,4.954158,0.011724,15.427580,41.323011,0.002908,0.143456,0.105782,1.226173,0.869685
min,4.400000,0.080000,0.000000,0.600000,0.012000,2.000000,19.000000,0.987110,2.790000,0.220000,8.400000,3.000000
25%,6.300000,0.210000,0.270000,1.762500,0.035000,23.000000,108.000000,0.991663,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.310000,5.300000,0.042500,34.000000,133.000000,0.993700,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.380000,9.900000,0.050000,45.000000,166.000000,0.996100,3.280000,0.540000,11.400000,6.000000
max,9.300000,0.580000,0.690000,20.800000,0.110000,86.000000,260.000000,1.001960,3.640000,0.830000,14.200000,9.000000


As we can see, our dataset has no outlier values after removing values with an absolute value z-score greater than or equal to 3. In total, there are now 1458 instances in the red wine dataset and 4502 instances in the white wine dataset. We also know that there are no missing values in either of these datasets from our analysis in Lab 1. We can now remove the quality and store it in a variable called "y" for each respective wine color. This process is shown below:

In [5]:
#extract prediction task (quality) from datasets and then remove
y_red = df_red['quality']
X_red = df_red.drop(columns='quality')
y_white = df_white['quality']
X_white = df_white.drop(columns='quality')

In [6]:
#data description dataframe, used in Lab 1
description_df = pd.DataFrame()

#df_white could also be used here since they have the same attributes
description_df['Attributes'] = df_red.columns

#Description of each attribute
description_df['Description'] = ['grams of tartaric acid per liter', 
                                 'grams of acetic acid per liter',
                                 'grams of citric acid per liter',
                                 'grams of sugar per liter remaining after fermentation stops',
                                 'grams of sodium chloride (salt) per liter',
                                 'milligrams of free form sulfur dioxide per liter',
                                 'milligrams of free form and bound forms of sulfur dioxide per liter',
                                 'density of the wine (grams per cubic centimeter)',
                                 'the pH value of the wine',
                                 'grams of potassium sulphate per liter',
                                 'percent of alcohol by volume',
                                 'median score given by wine tasters (prediction task)']

description_df['Scales'] = ['ratio', 'ratio', 'ratio', 'ratio', 
                            'ratio', 'ratio', 'ratio', 'ratio', 
                            'interval', 'ratio', 'ratio', 'ordinal']

description_df['Discrete\Continuous'] = ['Continuous', 'Continuous', 'Continuous',
                                        'Continuous', 'Continuous', 'Continuous',
                                        'Continuous', 'Continuous', 'Continuous',
                                        'Continuous', 'Continuous', 'Discrete']

#this display option found at: 
#https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin
pd.set_option('display.max_colwidth', None)
description_df

,Attributes,Description,Scales,Discrete\Continuous
0,fixed acidity,grams of tartaric acid per liter,ratio,Continuous
1,volatile acidity,grams of acetic acid per liter,ratio,Continuous
2,citric acid,grams of citric acid per liter,ratio,Continuous
3,residual sugar,grams of sugar per liter remaining after fermentation stops,ratio,Continuous
4,chlorides,grams of sodium chloride (salt) per liter,ratio,Continuous
5,free sulfur dioxide,milligrams of free form sulfur dioxide per liter,ratio,Continuous
6,total sulfur dioxide,milligrams of free form and bound forms of sulfur dioxide per liter,ratio,Continuous
7,density,density of the wine (grams per cubic centimeter),ratio,Continuous
8,pH,the pH value of the wine,interval,Continuous
9,sulphates,grams of potassium sulphate per liter,ratio,Continuous


__Note:__ this table/code was taken from our lab 1 assignment and slightly edited for our needs in this lab. 

The above table gives us a description of each feature, data type (scales), and whether it is discrete or continuous. It should be noted that the "quality" attribute is not actually in the "X_red" or "X_white", but we included it for clarity. This is the final dataset we will be using for our regression tasks. Both white and red wine datasets contain these attributes.

### 1.3 Split Data Into Training & Testing Sets (80/20 Split)

In [7]:
from sklearn.model_selection import train_test_split as tts

#perform 80/20 split on both datasets
X_train_r, X_test_r, y_train_r, y_test_r = tts(X_red, y_red, test_size=0.2)
X_train_w, X_test_w, y_train_w, y_test_w = tts(X_white, y_white, test_size=0.2)

We believe an 80/20 train-test split is a viable option for our dataset because of the size. We consider our datasets relatively small since our processed red wine dataset only contains 1458 instances and our processed white wine dataset contains 4502 instances. Because of this, we need as much training data as possible with a reasonable amount to test on. If our testing set was too small, we do not think the accuracy score of the model would truly represent the performance of the model. 

Another option to split data is dividing it into training, validation, and testing. Like our previous arguement, we do not think our datasets are large enough to perform this type of split. If we had tens to hundreds of thousands of instances, this type of split would be a possbility. 

## 2. Modeling
  
- [1.5 points] Train your classifier to achieve good generalization performance. That is, adjust the __optimization technique__ and the value of the __regularization term "C"__ to achieve the best performance on your test set. Visualize the performance of the classifier versus the parameters you investigated. Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?
- [1.5 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time and classification performance. __Discuss the results.__

### 2.1 Create Custom One Vs. All Logistic Regression Classifier

#### 2.1.1 Create BinaryLogisticRegression Class Using Example From Class

In [8]:
#code taken from 06. Optimization notebook
import numpy as np
from scipy.special import expit

class BinaryLogisticRegression:
    #extend init function by adding regularization term
    def __init__(self, eta, iterations=20, C=0.001, regularization_term='none'):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.reg_term = regularization_term
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    ################### private ###################
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    #Implement _get_gradient function in sub classes!
    
    ################### public ###################
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 
            # add bacause maximizing 

#### 2.1.2 Create SteepestDescentBLR Class Using Example From Class

In [9]:
class SteepestDescentBLR(BinaryLogisticRegression):
    
    # steepest descent calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        
        if self.reg_term == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        if self.reg_term == 'L1':
            gradient[1:] += np.sign(self.w_[1:]) * self.C
        if self.reg_term == 'both':
            gradient[1:] += (np.sign(self.w_[1:]) * self.C) + (-2 * self.w_[1:] * self.C)
        
        #if reg_term is 'none' no need to do anything, just return gradient
        return gradient

#### 2.1.3 Create StochasticGradientDescentBLR Class Using Example From Class

In [10]:
class StochasticGradientDescentBLR(BinaryLogisticRegression):
    
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        
        if self.reg_term == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        if self.reg_term == 'L1':
            gradient[1:] += np.sign(self.w_[1:]) * self.C
        if self.reg_term == 'both':
            gradient[1:] += (np.sign(self.w_[1:]) * self.C) + (-2 * self.w_[1:] * self.C)
        
        #if reg_term is 'none' no need to do anything, just return gradient
        return gradient

#### 2.1.4 Create NetwtonsMethodBLR Class Using Example From Class (Hessian)

In [11]:
from numpy.linalg import pinv

class NewtonsMethodBLR(BinaryLogisticRegression):
    
    # just overwrite gradient function
    def _get_gradient(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        
        if self.reg_term == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        if self.reg_term == 'L1':
            gradient[1:] += np.sign(self.w_[1:]) * self.C
        if self.reg_term == 'both':
            gradient[1:] += (np.sign(self.w_[1:]) * self.C) + (-2 * self.w_[1:] * self.C)
        
        #if reg_term is 'none' no need to do anything, just return gradient
        return pinv(hessian) @ gradient

#### 2.1.5 Create Multiclass Logistic Regression Class Using Example From Class

In [12]:
class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20, 
                 C=0.0001, 
                 solver=SteepestDescentBLR,
                 regularization_term='none'): #regularization_term options: 'none', 'L1', 'L2', 'both'
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        #store regularization term
        self.reg_term = regularization_term
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem
            # train the binary classifier for this class
            
            #add regularization term to solver params
            blr = self.solver(eta=self.eta,
                              iterations=self.iters,
                              C=self.C,
                              regularization_term=self.reg_term)
            blr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

To tackle creating a custom one-versus-all logistic regression classifier with the ability to choose different optimization techniques and custom regularizations, we first began with the code provided in notebook 06. optimization. We used the ```BinaryLogisticRegression``` class as our base class and extended it with the ```StochasticLogisticRegression``` (renamed to ```StochasticGradientDescentBLR```), ```HessianBinaryLogisticRegression``` (renamed to ```NewtonsMethodBLR```), and the ```MultiClassLogisticRegression``` classes. Our ```SteepestDescentBLR``` class uses the ```_get_gradient``` function from the base ```BinaryLogisticRegression``` class. We decided to remove the ```_get_gradient``` function from the base class as it is implemented in the ```SteepestDescentBLR``` class. In each of the ```BinaryLogisticRegression``` child classes, the only additional/changed function is the ```_get_gradient``` function which calculates the gradient based on the given optimization technique. 

In regards to adding the ability to choose different optimization techniques, we stuck with your implementation which allowed the classifier to be passed in via a parameter. To add the ability to change normalization term we added the ```regularization_term``` parameter to the base ```BinaryLogisticRegression``` class and the ```MultiClassLogisticRegression``` class. The value for this parameter can either be "L1", "L2", "both", or "none" which specifies the type of regularization to use. To implement the different regularizations, we added a chain of if statements in each child class's ```_get_gradient``` function. If the regularization term chosen was either "L1", "L2", or "both", the gradient was adjusted according to the regularization. If the regularization term was "none", the gradients were not adjusted and simply returned as is. 

### 2.2 Train Classifier to Acheive Good Generalization Performance

#### 2.2.1 Achieve Good Generalization Performance on Red Wine Dataset

In [13]:
import time
from sklearn.metrics import accuracy_score

solvers = [SteepestDescentBLR, StochasticGradientDescentBLR, NewtonsMethodBLR]
etas = [0.0001, 0.001, 0.01, 0.1, 1]
Cs = [0.0001, 0.001, 0.01, 0.1, 1]
regularization_terms = ['none', 'L1', 'L2', 'both']

#use various parameters to predict red wine dataset
best_acc_r = 0
best_params_r = {}
timing_r = 0
for solver in solvers:
    for eta in etas:
        for c in Cs:
            for reg_term in regularization_terms:
                #time each iteration
                t0 = time.time()
                #create the Multiclass Logistic Regression object, train, and test
                mclr = MultiClassLogisticRegression(eta=eta,
                                                    iterations=50,
                                                    C=c,
                                                    solver=solver,
                                                    regularization_term=reg_term)
                mclr.fit(X_train_r, y_train_r)
                mclr_yhat = mclr.predict(X_test_r)
                t1 = time.time()

                time_ms = (t1 - t0) * 1000
                acc = accuracy_score(y_test_r, mclr_yhat)
                
                if acc > best_acc_r:
                    best_acc_r = acc
                    best_params_r = {'Solver': solver,
                                     'Eta': eta,
                                     'C':c,
                                     'Regularization':reg_term}
                    timing_r = time_ms

print('Best model accuracy for red wine dataset is:', best_acc_r)
print('Best parameters are:', best_params_r)
print('Best model took %.2f ms to fit and predict' % timing_r)

Best model accuracy for red wine dataset is: 0.2534246575342466
Best parameters are: {'Solver': <class '__main__.StochasticGradientDescentBLR'>, 'Eta': 0.1, 'C': 0.1, 'Regularization': 'L2'}
Best model took 24.33 ms to fit and predict


As we can see, the best model for our red wine dataset has an accuracy score of around 25% in 24 milliseconds. It achieves this score by using the stochastic gradient descent optimazation, an eta value of 0.1, a C value of 0.1, and L2 regularization. In addition, this model was run with 50 iterations. Lets see if using these parameters and changing the number of iterations has any effect on the accuracy:

In [14]:
iters = [10, 20, 50, 100, 150, 200, 250, 300, 350, 400]

print('%10s %15s %8s' % ('Iterations', 'Accuracy', 'Time (ms)'))
for i in iters:
    #time each iteration
    t0 = time.time()
    #create the Multiclass Logistic Regression object, train, and test
    mclr = MultiClassLogisticRegression(eta=0.1,
                                        iterations=i,
                                        C=0.1,
                                        solver=StochasticGradientDescentBLR,
                                        regularization_term='L2')
    mclr.fit(X_train_r, y_train_r)
    mclr_yhat = mclr.predict(X_test_r)
    t1 = time.time()

    time_ms = (t1 - t0) * 1000
    acc = accuracy_score(y_test_r, mclr_yhat)
    
    print('%10d %0.13f %.2f' % (i, acc, time_ms))

Iterations        Accuracy Time (ms)
        10 0.0000000000000 29.69
        20 0.0000000000000 44.94
        50 0.0102739726027 59.14
       100 0.4486301369863 57.97
       150 0.0102739726027 86.23
       200 0.0034246575342 93.89
       250 0.0000000000000 99.96
       300 0.0000000000000 159.74
       350 0.0068493150685 141.35
       400 0.0034246575342 441.47


Look at that! Using 100 iterations this model was able to achieve 44% accuracy with a runtime of about 58 milliseconds. Lets visualize this models decision boundaries. To do this we will run PCA on our dataset to reduce it down to 2 dimensions:

In [ ]:
from sklearn import manifold, datasets

tsne = manifold.TSNE(n_components=2)
X_tsne = tsne.fit_transform(X_train_r)

from matplotlib import pyplot as plt
import copy
%matplotlib inline
plt.style.use('ggplot')

#copied from notebook 05. logistic regression
def plot_decision_boundaries(lr,Xin,y,title=''):
    Xb = copy.deepcopy(Xin)
    lr.fit(Xb[:,:2],y) # train only on two features

    h=0.01
    # create a mesh to plot in
    x_min, x_max = Xb[:, 0].min() - 1, Xb[:, 0].max() + 1
    y_min, y_max = Xb[:, 1].min() - 1, Xb[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # get prediction values
    Z = lr.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.5)

    # Plot also the training points
    plt.scatter(Xb[:, 0], Xb[:, 1], c=y, cmap=plt.cm.Paired)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(title)
    plt.show()
    
mclr = MultiClassLogisticRegression(eta=0.1,
                                    iterations=100,
                                    C=0.1,
                                    solver=StochasticGradientDescentBLR,
                                    regularization_term='L2')

plot_decision_boundaries(mclr,X_tsne,y_train_r)

#### 2.2.2 Achieve Good Generalization Performance on White Wine Dataset

In [29]:
#repeat same process but with white wine dataset
best_acc_w = 0
best_params_w = {}
timing_w = 0
for solver in solvers:
    for eta in etas:
        for c in Cs:
            for reg_term in regularization_terms:
                #time each iteration
                t0 = time.time()
                #create the Multiclass Logistic Regression object, train, and test
                mclr = MultiClassLogisticRegression(eta=1,
                                                    iterations=50,
                                                    C=c,
                                                    solver=solver,
                                                    regularization_term=reg_term)
                mclr.fit(X_train_w, y_train_w)
                mclr_yhat = mclr.predict(X_test_w)
                t1 = time.time()

                time_ms = (t1 - t0) * 1000
                acc = accuracy_score(y_test_w, mclr_yhat)

                if acc > best_acc_w:
                    best_acc_w = acc
                    best_params_w = {'Solver': solver,
                                     'Eta': eta,
                                     'C':c,
                                     'Regularization':reg_term}
                    timing_w = time_ms

print('Best model accuracy for white wine dataset is:', best_acc_w)
print('Best parameters are:', best_params_w)
print('Best model took %.2f ms to fit and predict' % timing_w)

Best model accuracy for white wine dataset is: 0.42508324084350724
Best parameters are: {'Solver': <class '__main__.StochasticGradientDescentBLR'>, 'Eta': 0.0001, 'C': 0.1, 'Regularization': 'both'}
Best model took 28.91 ms to fit and predict


As we can see, the best model for our white wine dataset has an accuracy score of around 42% in 28 milliseconds. It achieves this score by using the stochastic gradient descent optimazation, an eta value of 0.0001, a C value of 0.1, and both L1 and L2 regularizations. In addition, this model was run with 50 iterations. Lets see if using these parameters and changing the number of iterations has any effect on the accuracy:

In [48]:
iters = [10, 20, 50, 100, 150, 200, 250, 300, 350, 400]

print('%10s %15s %8s' % ('Iterations', 'Accuracy', 'Time (ms)'))
for i in iters:
    #time each iteration
    t0 = time.time()
    #create the Multiclass Logistic Regression object, train, and test
    mclr = MultiClassLogisticRegression(eta=0.0001,
                                        iterations=i,
                                        C=0.1,
                                        solver=StochasticGradientDescentBLR,
                                        regularization_term='both')
    mclr.fit(X_train_w, y_train_w)
    mclr_yhat = mclr.predict(X_test_w)
    t1 = time.time()

    time_ms = (t1 - t0) * 1000
    acc = accuracy_score(y_test_w, mclr_yhat)
    
    print('%10d %0.13f %.2f' % (i, acc, time_ms))

Iterations        Accuracy Time (ms)
        10 0.0000000000000 22.31
        20 0.0000000000000 21.46
        50 0.0000000000000 29.14
       100 0.0000000000000 38.64
       150 0.0000000000000 54.85
       200 0.0000000000000 92.95
       250 0.0188679245283 89.99
       300 0.0144284128746 107.19
       350 0.0011098779134 176.94
       400 0.0000000000000 290.06


### 2.3 Comparing Our Optimization Performance to The Godly Scikit-Learn Optimization

#### 2.3.1 Comparing Scikit-Learn Optimization on Red Wine Dataset

In [54]:
from sklearn.linear_model import LogisticRegression

#test sklearn on red wine dataset
t0 = time.time()
skl_logreg = LogisticRegression(solver='liblinear')

skl_logreg.fit(X_train_r, y_train_r)
skl_yhat = skl_logreg.predict(X_test_r)
t1 = time.time()

print('SciKit-Learn accuracy score on red wine test dataset:', 
      accuracy_score(y_test_r, skl_yhat))
print('Total execution time: %.2f milliseconds' % ((t1-t0) * 1000))

SciKit-Learn accuracy score on red wine test dataset: 0.6095890410958904
Total execution time: 40.13 milliseconds


#### 2.3.2 Comparing Scikit-Learn Optimization on White Wine Dataset

In [55]:
#test sklearn on white wine dataset
t0 = time.time()
skl_logreg = LogisticRegression(solver='liblinear')

skl_logreg.fit(X_train_w, y_train_w)
skl_yhat = skl_logreg.predict(X_test_w)
t1 = time.time()

print('SciKit-Learn accuracy score on white wine test dataset:', 
      accuracy_score(y_test_w, skl_yhat))
print('Total execution time: %.2f milliseconds' % ((t1-t0) * 1000))

SciKit-Learn accuracy score on white wine test dataset: 0.5338512763596004
Total execution time: 152.18 milliseconds


## 3. Deployment

- Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

__TODO:__ why our model sucks

## 4. Exceptional Work: 

- You have free reign to provide additional analyses. __One idea:__ Update the code to use either "one-versus-all" or "one-versus-one" extensions of binary to multi-class classification. 
- __Required for 7000 level students:__ Choose ONE of the following:
    1. __Option One:__ Implement an optimization technique for logistic regression using __mean square error__ as your objective function (instead of maximum likelihood). Derive the gradient updates for the Hessian and use Newton's method to update the values of "w". Then answer, which process do you prefer: maximum likelihood OR minimum mean-squared error? 
    2. __Option Two:__ Implement the BFGS algorithm from scratch to optimize logistic regression. That is, use BFGS without the use of an external package (for example, do not use SciPy). Compare your performance accuracy and runtime to the BFGS implementation in SciPy (that we used in lecture). 

## References

[1] UCI Machine Learning Repository. Wine Quality Dataset. https://archive.ics.uci.edu/ml/datasets/Wine+Quality

[2] Lab 1: Exploring Table Data